In [1]:
using PlutoUI
	using CSV
	using DataFrames
	using StatsPlots
	using Plots
	using Statistics

In [2]:
df = CSV.read(joinpath(dirname(@__FILE__), "Concrete_Data_DNO.csv"), DataFrame)

,Cement,Blast_Furnace_Slag,Fly_Ash,Water,Superplasticizer,Coarse_Aggregate
,Float64,Float64,Float64,Float64,Float64,Float64
1,540.0,0.0,0.0,162.0,2.5,1040.0
2,540.0,0.0,0.0,162.0,2.5,1055.0
3,332.5,142.5,0.0,228.0,0.0,932.0
4,332.5,142.5,0.0,228.0,0.0,932.0
5,198.6,132.4,0.0,192.0,0.0,978.4
6,266.0,114.0,0.0,228.0,0.0,932.0
7,380.0,95.0,0.0,228.0,0.0,932.0
8,380.0,95.0,0.0,228.0,0.0,932.0
9,266.0,114.0,0.0,228.0,0.0,932.0


In [3]:
#splitting into 70% training, 15% test, 15% validation

x = Matrix(df[:, 1:8])
y = Matrix(df[:, 9:9])

xtrain = x[1:700, :]
ytrain = y[1:700]

xtest = x[701:850, :]
ytest = y[701:850]

xval = x[851:1030, :]
yval = y[851:1030]

180-element Vector{Float64}:
 37.36
 32.84
 42.64
 40.06
 41.94
 61.23
 40.87
 33.3
 52.42
 15.09
  ⋮
 31.88
 41.54
 39.46
 37.92
 44.28
 31.18
 23.7
 32.77
 32.4

In [4]:
function mse(ŷ::Vector{T}, y::Vector{T})::T where T<:AbstractFloat
	sum((ŷ-y).^2/length(ŷ))
end

mse (generic function with 1 method)

In [5]:
function minimize!(f_model::Function, x::Matrix{T}, y::Vector{T}, p::Vector{T}, η::T, num_steps::Int)::Vector{T} where T<:AbstractFloat
	for step in 1:num_steps
		g(p) = mse(f_model(x, p), y)
		p .-= η.* g'(p)
	end
	p
end

minimize! (generic function with 1 method)

In [6]:
function model(x::Matrix{T}, p::Vector{T})::Vector{T} where T<:AbstractFloat
	sum(x.*p[1:(length(p)-1)]', dims=2)[:].+p[length(p)]
end

model (generic function with 1 method)

In [8]:
#begin
	#df2 = combine(groupby(df, :Age), [:Blast_Furnace_Slag, :Fly_Ash, :Water, :Superplasticizer, :Coarse_Aggregate] => ((a, b, c, d, e) -> (Blast_Furnace_Slag=mean(a), Fly_Ash=mean(b), Water=mean(c), Superplasticizer=mean(d), Coarse_Aggregate=mean(e))) => AsTable)
	#yy = Vector(combine(groupby(df, :Age), :Concrete_compressive_strength => sum)[:, 2])
#end

In [24]:
function predict_transmission(df::DataFrame)
	pp = minimize!(model, xtrain, ytrain, rand(6), 0.01, 10000)
	model(Matrix(df[:, 2:end]), pp)
end

predict_transmission (generic function with 2 methods)

In [10]:
#RMSE= round((sum((yy .- predict_transmission(df2)).^2) / length(yy))^0.5, digits=1)

In [26]:
#predict_transmission(df)

In [18]:
#plotting graph to see predicted vs true
compare_plt = plot(ytest, predict_transmission(df), 
st=:scatter,
xlims=(0,80),
ylims=(0,80),
xlab = "True Value (ytest)",
ylab = "Predicted Value (y_pr)",
size = (500,500))

xx = 0:80
yy = 0:80

plot!(compare_plt, xx, yy, st=:line)

mse_SVR = mse(y_pr, ytest)
annotate!(15,75, text("mse = $(round(mse_SVR, sigdigits=2))", 10))

display(compare_plt)

MethodError: MethodError: no method matching adjoint(::var"#g#9"{typeof(model), Matrix{Float64}, Vector{Float64}})
Closest candidates are:
  adjoint(!Matched::Union{LinearAlgebra.QR, LinearAlgebra.QRCompactWY, LinearAlgebra.QRPivoted}) at C:\Users\user\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\LinearAlgebra\src\qr.jl:517
  adjoint(!Matched::Union{LinearAlgebra.Cholesky, LinearAlgebra.CholeskyPivoted}) at C:\Users\user\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\LinearAlgebra\src\cholesky.jl:558
  adjoint(!Matched::LinearAlgebra.Rotation) at C:\Users\user\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\LinearAlgebra\src\givens.jl:58
  ...